In [ ]:
import os
import pandas as pd
pd.set_option("display.max_columns", 50)

In [ ]:
username = os.environ['JUPYTERHUB_USER']
hiveaddr = os.environ['HIVE_SERVER_2']
print("Operating as: {0}".format(username))

In [ ]:
from pyhive import hive

conn = hive.connect(host=hiveaddr, 
                    port=10000,
                    username=username) 
cur = conn.cursor()

In [ ]:
query = """
    select * from {0}.timetable_trips_csv limit 5
""".format(username)
pd.read_sql(query, conn)(47.378177, 8.540192)

In [ ]:
query = """
    select * from {0}.geostops_orc
    where cast(STOP_LAT as decimal(19, 9)) > 0
    limit 5
""".format(username)
pd.read_sql(query, conn)